In [73]:
! git clone https://github.com/kristerseiya/Plug-and-Play-Priors.git

Cloning into 'Plug-and-Play-Priors'...
remote: Enumerating objects: 293, done.
remote: Counting objects: 100% (293/293), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 293 (delta 151), reused 203 (delta 72), pack-reused 0
Receiving objects: 100% (293/293), 14.95 MiB | 1.57 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [74]:
% cd Plug-and-Play-Priors

/content/Plug-and-Play-Priors


In [82]:
! git pull origin main

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/kristerseiya/Plug-and-Play-Priors
 * branch            main       -> FETCH_HEAD
   c3dff05..ef8aae3  main       -> origin/main
Updating c3dff05..ef8aae3
Fast-forward
 DnCNN/run.py | 2 --
 1 file changed, 2 deletions(-)


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [129]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import MultiStepLR, ExponentialLR
from importlib import reload

In [6]:
from DnCNN import data
from DnCNN import model
from DnCNN import run

In [108]:
reload(data)
reload(model)
reload(run)

<module 'DnCNN.run' from '/content/Plug-and-Play-Priors/DnCNN/run.py'>

In [147]:
gdrive_root = '/content/gdrive/MyDrive/Dataset/'
data_dirs = [gdrive_root+'DIV2K_train_HR/DIV2K_train_HR', gdrive_root+'DIV2K_val_HR/DIV2K_val_HR']
dataset = data.ImageDataset(data_dirs, store='ram', repeat=10, resize=-1)
trainset, valset, testset = dataset.split(0.7, 0.1, 0.2)

In [148]:
trainldr = DataLoader(trainset, batch_size=64, shuffle=True, drop_last=True)
valldr = DataLoader(valset, batch_size=64, shuffle=False)
testldr = DataLoader(testset, batch_size=1, shuffle=False)

In [149]:
device = torch.device('cuda')

In [162]:
net = model.DnCNN(act_mode='BR').move(device)
# optimizer = SGD(net.parameters(), lr=1e-2, weight_decay=1e-4, momentum=0.9)
# scheduler = ExponentialLR(optimizer, 1e-3 ** (1 / 50) )
optimizer = Adam(net.parameters(), lr=1e-3)
scheduler = MultiStepLR(optimizer, [25,], gamma=0.1)

In [163]:
log = run.train(net, optimizer, 40, trainldr, 50, validation=valldr, scheduler=scheduler)


Epoch #1
Train Loss: 0.03264
Val Loss: 0.00941

Epoch #2
Train Loss: 0.00807
Val Loss: 0.00505

Epoch #3
Train Loss: 0.00629
Val Loss: 0.00587

Epoch #4
Train Loss: 0.00489
Val Loss: 0.00529

Epoch #5
Train Loss: 0.00448
Val Loss: 0.00328

Epoch #6
Train Loss: 0.00389
Val Loss: 0.00378

Epoch #7
Train Loss: 0.00415
Val Loss: 0.00794

Epoch #8
Train Loss: 0.00383
Val Loss: 0.00354

Epoch #9
Train Loss: 0.00355
Val Loss: 0.00374

Epoch #10
Train Loss: 0.00362
Val Loss: 0.00338

Epoch #11
Train Loss: 0.00327
Val Loss: 0.00603

Epoch #12
Train Loss: 0.00320
Val Loss: 0.00293

Epoch #13
Train Loss: 0.00306
Val Loss: 0.00416

Epoch #14
Train Loss: 0.00307
Val Loss: 0.00384

Epoch #15
Train Loss: 0.00310
Val Loss: 0.00284

Epoch #16
Train Loss: 0.00295
Val Loss: 0.00310

Epoch #17
Train Loss: 0.00295
Val Loss: 0.00284

Epoch #18
Train Loss: 0.00291
Val Loss: 0.00297

Epoch #19
Train Loss: 0.00286
Val Loss: 0.00370

Epoch #20
Train Loss: 0.00280
Val Loss: 0.00362

Epoch #21
Train Loss: 0.0028

In [166]:
test_mse = run.validate(net, testldr, 50)
print('MSE: {:.5f}'.format(test_mse))

MSE: 0.00242


In [165]:
torch.save(net.state_dict(), 'dncnn50.pth')